In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from PIL import Image
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask

In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")


GPU found


In [8]:
def CapsNet(input_shape, n_class, routings, batch_size,dim):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :param batch_size: size of batch
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x2 = layers.Input(shape=input_shape, batch_size=batch_size)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv1D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x2)

    conv1 = tf.reshape(conv1,[1,100,3022,256])
    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings, name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
   # out_caps = Length(name='capsnet')(digitcaps)
    
    # Decoder network.
   # y = layers.Input(shape=(n_class,))
    #masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    normal_cont = tf.reshape(digitcaps, [-1,32])
    print(normal_cont.shape)
  #  masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction
    
   # print(masked_by_y.shape)
  #  print(masked.shape)
    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu',input_dim=32))
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    ########################################################################################################
    vocab_size = 30  #after K-fold , seems the size here will not make a significant difference on accuracy
    maxlen = 3030  # length of input
    embed_dim = 32+32*dim  # Embedding size for each token
    num_heads = 2   # Number of attention heads
    ff_dim = 128  # Hidden layer size in feed forward network inside transformer

    inputs = decoder(normal_cont)   
    inputs = layers.Reshape((3030,))(inputs)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Reshape((352, 1))(x)

    x = layers.Conv1D(filters=32, kernel_size=2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(filters=32, kernel_size=2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(200, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    train_model = keras.Model(inputs=x2, outputs=outputs)
    # Models for training and evaluation (prediction)
    #train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    #eval_model = models.Model(x, [out_caps, decoder(masked)])

    # manipulate model
    #noise = layers.Input(shape=(n_class, 16))
    #noised_digitcaps = layers.Add()([digitcaps, noise])
    #masked_noised_y = Mask()([noised_digitcaps, y])
    #manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model


def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    # return tf.reduce_mean(tf.square(y_pred))
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))

    return tf.reduce_mean(tf.reduce_sum(L, 1))


In [4]:
#Transformer Structure
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [5]:
xtrain = r"NEWdata/NEW_training_sample.csv.gz"
ytrain = r"NEWdata/NEW_training_label.csv.gz"
xtest = r"NEWdata/NEW_testing_sample.csv.gz"
ytest = r"NEWdata/NEW_testing_label.csv.gz"

i=0
samplesdf = pd.read_csv(xtrain,compression ="gzip",delimiter=',')
x_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytrain,compression ="gzip",delimiter=',')
y_trai = samplesdf.to_numpy()
y_train = np.zeros((x_train.shape[0],2))
for yt in y_trai:
    if yt == 0:
        y_train[i] = [1,0]
    else:
        y_train[i] = [0,1]
    i = i + 1

samplesdf = pd.read_csv(xtest,compression ="gzip",delimiter=',')
x_test = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytest,compression ="gzip",delimiter=',')
y_tes = samplesdf.to_numpy()
y_test = np.zeros((x_test.shape[0],2))
i = 0
for yt in y_tes:
    if yt == 0:
        y_test[i] = [1,0]
    else:
        y_test[i] = [0,1]
    i = i + 1


print("done")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
y_test

done
(38483, 3030)
(38483, 2)
(4265, 3030)
(4265, 2)


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [6]:
def gen_model(dim):
#hyper parameter
    vocab_size = 30  #after K-fold , seems the size here will not make a significant difference on accuracy
    maxlen = 3148  # length of input
    embed_dim = 32+32*dim  # Embedding size for each token
    num_heads = 2   # Number of attention heads
    ff_dim = 128  # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Reshape((352, 1))(x)

    x = layers.Conv1D(filters=32, kernel_size=2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(filters=32, kernel_size=2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(200, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)



    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
    return model

In [9]:
with tf.device('/CPU:0'):
    model2 = CapsNet(input_shape=(3030,), #need change
                                        n_class=2,
                                        routings=3,
                                        batch_size=100,dim=10)

    model2.compile(optimizer=optimizers.Adam(lr=1),
                                  loss=[margin_loss, 'mse'],
                                  loss_weights=[1., 2])
model2.summary()

#model=gen_model(10)
#model.summary()

(1, 32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(100, 3030, 1)]          0         
_________________________________________________________________
conv1 (Conv1D)               (100, 3022, 256)          2560      
_________________________________________________________________
tf.reshape_2 (TFOpLambda)    (1, 100, 3022, 256)       0         
_________________________________________________________________
primarycap_conv1d (Conv1D)   (1, 100, 1507, 256)       590080    
_________________________________________________________________
primarycap_reshape (Reshape) (1, 4822400, 8)           0         
_________________________________________________________________
primarycap_squash (Lambda)   (1, 4822400, 8)           0         
_________________________________________________________________
digitcaps (CapsuleLayer)     (2, 16)                 

In [56]:
model2.fit(x_train,y_train,batch_size=100,epochs=10)

Epoch 1/10


TypeError: 'NoneType' object is not callable

In [4]:
#Result analysis
testresult=model.predict(x_test)

i = 0
correct = 0
for x in testresult:
    if x >=0.5 and y_test[i] == 1:
        correct = correct + 1
    elif x < 0.5 and y_test[i] == 0:
        correct = correct + 1
    i = i + 1
testacc = correct/i
testacc

fpr, tpr, _ = roc_curve(y_test,testresult)
roc_auc = auc(fpr,tpr)
plt.figure()
lw = 2
plt.plot(fpr,tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUROC curve for Series traditional Transformer + CNN')
plt.legend(loc="lower right")
plt.savefig("/uac/cprj/cprj2716/borrow_from_ESETR_gpmate_tokaho/3sTransformerAUROC.png")

plt.figure()
precision, recall, _ = precision_recall_curve(y_test,testresult)
prc_auc = auc(recall,precision)
plt.plot(recall,precision, color='darkorange',
         lw=lw, label='PRC curve (area = %0.2f)' % prc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC curve for Series traditional Transformer + CNN')
plt.legend(loc="lower right")
plt.savefig("/uac/cprj/cprj2716/borrow_from_ESETR_gpmate_tokaho/3sTransformerAUPRC.png")
print("AUPRC = %.02f"% prc_auc)
ss = np.zeros((len(testresult)))
i = 0
for x in testresult:
    if x >= 0.5:
        ss[i] = 1
    else:
        ss[i] = 0
    i = i + 1
f1s = f1_score(y_test,ss)
print("f1_score = %.02f"% f1s)

model.save('/uac/cprj/cprj2716/borrow_from_ESETR_gpmate_tokaho/seriesTrCNN_3s')

NameError: name 'model' is not defined